In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import time

In [ ]:
train_on_gpu = torch.cuda.is_available()
train_on_gpu=False
if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

In [ ]:
transforms.Resize([299, 299]),

In [ ]:
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler

# number of subprocesses to use for data loading
num_workers = 0
# how many samples per batch to load
batch_size = 20
# percentage of training set to use as validation
valid_size = 0.2

# convert data to a normalized torch.FloatTensor
transform = transforms.Compose([
    transforms.Resize([299, 299]),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

# choose the training and test datasets
train_data = datasets.CIFAR10('data', train=True,
                              download=True, transform=transform)
test_data = datasets.CIFAR10('data', train=False,
                             download=True, transform=transform)

# obtain training indices that will be used for validation
num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# define samplers for obtaining training and validation batches
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# prepare data loaders (combine dataset and sampler)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,
    sampler=train_sampler, num_workers=num_workers)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, 
    sampler=valid_sampler, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, 
    num_workers=num_workers)

# specify the image classes
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

In [ ]:
class Xception_implementation(nn.Module):
    def __init__(self):
        super(Xception_implementation,self).__init__()

        self.c1=nn.Conv2d(3,32,3,stride=2)
        self.bn1=nn.BatchNorm2d(32)

        self.c2=nn.Conv2d(32,64,3)
        self.bn2=nn.BatchNorm2d(64)

        self.c3=nn.Conv2d(64,128,1,stride=2)
        self.bn3=nn.BatchNorm2d(128)

        self.c4=nn.Conv2d(128,256,1,stride=2)
        self.bn4=nn.BatchNorm2d(256)

        self.c5=nn.Conv2d(256,728,1,stride=2)
        self.bn5=nn.BatchNorm2d(728)

        self.c6=nn.Conv2d(728,728,1,stride=2)
        #self.bn5=nn.BatchNorm2d(728)

        self.c7=nn.Conv2d(728,1024,1,stride=2)
        self.bn6=nn.BatchNorm2d(1024)


        self.sc1a=nn.Conv2d(64,64,3,groups=64,padding=1,bias=False)
        self.sc1b=nn.Conv2d(64,128,1,bias=False)
        #self.bn3=nn.BatchNorm2d(128)

        self.sc2a=nn.Conv2d(128,128,3,groups=128,padding=1,bias=False)
        self.sc2b=nn.Conv2d(128,128,1,bias=False)
        #self.bn3=nn.BatchNorm2d(128)

        self.sc3a=nn.Conv2d(128,128,3,groups=128,padding=1,bias=False)
        self.sc3b=nn.Conv2d(128,256,1,bias=False)
        #self.bn4=nn.BatchNorm2d(256)

        self.sc4a=nn.Conv2d(256,256,3,groups=256,padding=1,bias=False)
        self.sc4b=nn.Conv2d(256,256,1,bias=False)
        #self.bn4=nn.BatchNorm2d(256)

        self.sc5a=nn.Conv2d(256,256,3,groups=256,padding=1,bias=False)
        self.sc5b=nn.Conv2d(256,728,1,bias=False)
        #self.bn5=nn.BatchNorm2d(728)

        self.sc6a=nn.Conv2d(728,728,3,groups=728,padding=1,bias=False)
        self.sc6b=nn.Conv2d(728,728,1,bias=False)
        #self.bn5=nn.BatchNorm2d(728)

        self.sc7a=nn.Conv2d(728,728,3,groups=728,padding=1,bias=False)
        self.sc7b=nn.Conv2d(728,1024,1,bias=False)
        #self.bn6=nn.BatchNorm2d(1024)

        self.sc8a=nn.Conv2d(1024,1024,3,groups=1024,padding=1,bias=False)
        self.sc8b=nn.Conv2d(1024,1536,1,bias=False)
        self.bn7=nn.BatchNorm2d(1536)

        self.sc9a=nn.Conv2d(1536,1536,3,groups=1536,padding=1,bias=False)
        self.sc9b=nn.Conv2d(1536,2048,1,bias=False)
        self.bn8=nn.BatchNorm2d(2048)

        self.fc1=nn.Linear(2048,512)
        self.ol=nn.Linear(512,10)


        self.maxpool=nn.MaxPool2d(kernel_size=(3,3),stride=2,padding=1)
        self.avgpool=nn.AvgPool2d(kernel_size=(10,10))
        


    def forward(self,x):
        x=self.c1(x)
        x=self.bn1(x)
        x=F.relu(x)

        x=self.c2(x)
        x=self.bn2(x)
        x=F.relu(x)
        output=self.c3(x)
        output=self.bn3(output)

        x=self.sc1b(self.sc1a(x))
        x=self.bn3(x)
        x=F.relu(x)
        x=self.sc2b(self.sc2a(x))
        x=self.bn3(x)
        x=self.maxpool(x)

        x=x+output
        output=self.c4(output)
        output=self.bn4(output)

        x=self.sc3b(self.sc3a(F.relu(x)))
        x=self.bn4(x)
        x=self.sc4b(self.sc4a(F.relu(x)))
        x=self.bn4(x)
        x=self.maxpool(x)

        x=x+output
        output=self.c5(output)
        output=self.bn5(output)
        
        x=self.sc5b(self.sc5a(F.relu(x)))
        x=self.bn5(x)
        x=self.sc6b(self.sc6a(F.relu(x)))
        x=self.bn5(x)
        x=self.maxpool(x)

        x=x+output

        output=x
        for _ in range(8):
            x=self.sc6b(self.sc6a(F.relu(x)))
            x=self.bn5(x)
            x=self.sc6b(self.sc6a(F.relu(x)))
            x=self.bn5(x)
            x=self.sc6b(self.sc6a(F.relu(x)))
            x=self.bn5(x)
            x=x+output
        
        output=self.c7(x)
        output=self.bn6(output)

        x=self.sc6b(self.sc6a(F.relu(x)))
        x=self.bn5(x)
        x=self.sc7b(self.sc7a(F.relu(x)))
        x=self.bn6(x)
        x=self.maxpool(x)

        x=x+output

        x=self.sc8b(self.sc8a(x))
        x=self.bn7(x)
        x=F.relu(x)
        x=self.sc9b(self.sc9a(x))
        x=self.bn8(x)
        x=F.relu(x)

        x=self.avgpool(x)
        x = x.view(x.size(0), -1) 

        x=F.relu(self.fc1(x))
        x=F.relu(self.ol(x))
        return x


model = Xception_implementation()
print(model)

# move tensors to GPU if CUDA is available
if train_on_gpu:
    model.cuda()

In [ ]:
import torch.optim as optim

# specify loss function
criterion = nn.CrossEntropyLoss()

# specify optimizer
optimizer = optim.SGD(model.parameters(),lr=0.0005, momentum=0.9)

In [ ]:
n_epochs = 1 
valid_loss_min = np.Inf # track change in validation loss
batch_number=0
for epoch in range(1, n_epochs+1):

    # keep track of training and validation loss
    train_loss = 0.0
    valid_loss = 0.0
    
    ###################
    # train the model #
    ###################
    model.train()
    
    for data, target in train_loader:
        batch_number+=1
        print(batch_number)
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # clear the gradients of all optimized variables
        optimizer.zero_grad()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss

        
        loss = criterion(output, target)

        # backward pass: compute gradient of the loss with respect to model parameters
        loss.backward()
        # perform a single optimization step (parameter update)
        optimizer.step()
        # update training loss
        train_loss += loss.item()*data.size(0)

    ######################    
    # validate the model #
    ######################
    model.eval()
    for data, target in valid_loader:
        # move tensors to GPU if CUDA is available
        if train_on_gpu:
            data, target = data.cuda(), target.cuda()
        # forward pass: compute predicted outputs by passing inputs to the model
        output = model(data)
        # calculate the batch loss
        loss = criterion(output, target)
        # update average validation loss 
        valid_loss += loss.item()*data.size(0)
    
    # calculate average losses
    train_loss = train_loss/len(train_loader.dataset)
    valid_loss = valid_loss/len(valid_loader.dataset)
        
    # print training/validation statistics 
    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
        epoch, train_loss, valid_loss))
    
    # save model if validation loss has decreased
    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
        valid_loss_min,
        valid_loss))
        torch.save(model.state_dict(), 'model_cifar.pt')
        valid_loss_min = valid_loss

In [ ]:
test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()
# iterate over test data
for data, target in test_loader:
    # move tensors to GPU if CUDA is available
    if train_on_gpu:
        data, target = data.cuda(), target.cuda()
    # forward pass: compute predicted outputs by passing inputs to the model
    output = model(data)
 
    # calculate the batch loss
    loss = criterion(output, target)
    # update test loss 
    test_loss += loss.item()*data.size(0)
    # convert output probabilities to predicted class
    _, pred = torch.max(output, 1)    
    # compare predictions to true label
    correct_tensor = pred.eq(target.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    # calculate test accuracy for each object class
    for i in range(batch_size):
        label = target.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

# average test loss
test_loss = test_loss/len(test_loader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(10):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))